In [31]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Load the dataset
data = pd.read_csv("./Data/2018-06-06-ss.cleaned.csv")

sequences = data.iloc[:, 2].values
structures = data.iloc[:, 4].values
max_len = 5037


In [32]:
# Convert the structures to classes
structures_classes = {" ": 0, "E": 1, "C": 2, "H": 3}
#Make the data the same length
for i in range(len(sequences)):
    sequences[i] = sequences[i] + " " * (max_len - len(sequences[i]))
    structures[i] = structures[i] + " " * (max_len - len(structures[i]))

# Encode the sequences and structures
for i in range(len(sequences)):
    sequences[i] = [ord(x) for x in sequences[i]]

print(sequences)
print(structures) 


In [ ]:
# Define the model architecture
class SecondaryStructureModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SecondaryStructureModel, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, output_size),
        )

    def forward(self, x):
        return self.net(x)


In [ ]:
# Define the hyperparameters and optimizer
input_size = 1
hidden_size = 64
output_size = 3
lr = 0.01
epochs = 10

model = SecondaryStructureModel(input_size, hidden_size, output_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

In [ ]:
# Training loop
for epoch in range(epochs):
    model.train()
    for i in range(len(sequences)):
        x = torch.tensor(sequences[i]).unsqueeze(0)
        y = torch.tensor(structures[i]).unsqueeze(0)
        optimizer.zero_grad()
        output = model(x)
        predicted = torch.softmax(output, dim=0).argmax(dim=0)
        loss = criterion(predicted, y)
        loss.backward()
        optimizer.step()

TypeError: new(): invalid data type 'str'

In [ ]:
    # Validation loop
    model.eval()
    with torch.inference_mode():

        total = 0
        correct = 0
        for i in range(len(sequences)):
            x = torch.tensor(sequences[i]).unsqueeze(0)
            y = torch.tensor(structures[i]).unsqueeze(0)
            output = model(x)
            predicted = torch.softmax(output, dim=1).argmax(dim=1)
            total += y.shape[1]
            correct += (predicted == y).sum().item()

        accuracy = correct / total
        print(f"Test Accuracy: {accuracy:.4f}")

RuntimeError: expected scalar type Long but found Float